# Raw Corpora Intake - One Off

With some shortcuts since I'm mostly pulling from HermitDave's GitHub stash.

## Imports and Settings

In [1]:
from nate_givens_toolkit import cloud_io as cloud
# from nate_givens_toolkit import local_io as local
import pandas as pd
from datetime import datetime
#import requests
#import boto3

## Global Variables

In [2]:
LOCAL_DIR = '/data/lexgen/raw_corpora/'
BUCKET_DIR = 'raw_corpora/'
BUCKET = 'lexgen'

## Logic

### Specify Raw Corpora Variables

In [3]:
url_filename = 'en_full.txt'
url_path = 'https://raw.githubusercontent.com/hermitdave/FrequencyWords/master/content/2018/en/'
filename = 'en_full_2018.txt'
lang_code = 'en'
source_url = 'https://raw.githubusercontent.com/hermitdave/FrequencyWords/master/content/2018/en/en_full.txt'
last_load_dtime = datetime.utcnow()
note = 'HermitDave\'s version of the full 2018 English file from OpenSubtitles'


### Read In Raw Corpora Table

In [4]:
raw_corpora_inventory = cloud.read_csv_from_s3('raw_corpora_inventory.dat', 'data_files/', BUCKET, sep='|')

In [5]:
raw_corpora_inventory.head()

,filename,lang_code,source_url,last_load_dtime,note


### Check raw_corpora_inventory for Pre-existing Duplicates

In [6]:
can_proceed = True

In [7]:
if filename in raw_corpora_inventory['filename'].unique():
    can_proceed = False
    print(f'filename {filename} already exists in raw_corpora_inventory.')

In [8]:
if source_url in raw_corpora_inventory['source_url'].unique():
    can_proceed = False
    print(f'source_url {source_url} already exists in raw_corpora_inventory.')

### Copy File Locally

In [9]:
if can_proceed:
    cloud.pull_data_from_url(url_filename, url_path, LOCAL_DIR, local_filename=filename, overwrite=True)

### Copy File to S3

In [10]:
if can_proceed:
    cloud.push_file_to_s3(filename, LOCAL_DIR, BUCKET_DIR, BUCKET, overwrite=True)

### Update raw_corpora_inventory

In [11]:
new_row = {'filename': filename, 'lang_code':lang_code, 'source_url':source_url, 'last_load_dtime':str(last_load_dtime), 'note':note}

In [12]:
raw_corpora_inventory = raw_corpora_inventory.append(new_row, ignore_index=True)

In [13]:
raw_corpora_inventory.head()

,filename,lang_code,source_url,last_load_dtime,note
0,en_full_2018.txt,en,https://raw.githubusercontent.com/hermitdave/F...,2021-03-26 02:04:07.270824,HermitDave's version of the full 2018 English ...


### Write Raw Corpora Back to S3

In [15]:
cloud.write_csv_to_s3('raw_corpora_inventory.dat', BUCKET_DIR, BUCKET, raw_corpora_inventory, sep='|', index=False)